In [1]:
# create the configuration of the low-fidelity model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

# get the accuracy metrics
from sklearn.metrics import r2_score

from mfbml.metrics import (
    mean_log_likelihood_value,
    normalized_mae,
    normalized_rmse,
)
from mfbml.methods.bayes_neural_nets import BNNWrapper
from mfpml.models.hierarchical_kriging import HierarchicalKriging

from mfpml.models.gaussian_process import GaussianProcessRegression
# fix the random seed for reproducibility
seed = 1999
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
# read the data
data = pd.read_pickle("data.pkl")
# extract dataset
samples = data["samples"]
hf_samples = samples["hf"]
lf_samples = samples["lf"]
hf_responses = data["responses_lf1"]["hf"]
responses_lf1 = data["responses_lf1"]
responses_lf2 = data["responses_lf2"]
responses_lf3 = data["responses_lf3"]
test_samples = data["test_samples"]
test_hf_responses_noiseless = data["test_hf_responses_noiseless"]
test_hf_responses = data["test_hf_responses"]
test_lf1_responses = data["test_lf1_responses"]
test_lf2_responses = data["test_lf2_responses"]
test_lf3_responses = data["test_lf3_responses"]

In [3]:
# normalize the responses
hf_responses_mean = hf_responses.mean().numpy()
hf_responses_std = hf_responses.std().numpy()
hf_responses_scaled = (hf_responses.clone() -
                       hf_responses_mean) / hf_responses_std
sigma_scale = float(0.05 / hf_responses_std)
# create the sf_bnn model
for ii in [50, 128, 256, 512, 1024]:

    bnn_model = BNNWrapper(
        in_features=1,
        hidden_features=[ii, ii],
        out_features=1,
        activation="Tanh",
        lr=0.001,
        sigma=sigma_scale,
    )
    # train the model
    bnn_model.train(
        x=hf_samples,
        y=hf_responses_scaled,
        num_epochs=30000,
        sample_freq=100,
        burn_in_epochs=20000,
    )
    # save the model
    with open(f"sf_bnn_model_{ii}_{ii}.pkl", "wb") as f:
        torch.save(bnn_model, f)

    (bnn_y,
     bnn_epistemic,
     bnn_total_unc,
     bnn_aleatoric) = bnn_model.predict(
        x=test_samples)
    bnn_y = bnn_y * hf_responses_std + hf_responses_mean
    bnn_total_unc = bnn_total_unc * hf_responses_std
    bnn_epistemic = bnn_epistemic * hf_responses_std
    bnn_aleatoric = bnn_aleatoric * hf_responses_std
    plt.figure()
    plt.plot(hf_samples, hf_responses, "o", label="hf")
    plt.plot(test_samples.numpy(), bnn_y, label="hf prediction")

    plt.plot(
        test_samples.numpy(),
        test_hf_responses_noiseless.numpy(),
        label="hf ground truth",
    )
    plt.fill_between(
        test_samples.flatten().numpy(),
        (bnn_y - 2 * bnn_total_unc).reshape(-1),
        (bnn_y + 2 * bnn_total_unc).reshape(-1),
        alpha=0.5,
        label="uncertainty",
    )
    # set the ylim
    plt.ylim([-1.75, 1.75])
    plt.legend()
    # plt.show()
    plt.savefig(f"sf_bnn_{ii}_{ii}.png")
    plt.close()

/home/yaga/Documents/mfbml/src/mfbml/inference/psgld.py:93: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1519.)
  V.mul_(alpha).addcmul_(1 - alpha, d_p, d_p)


epoch:     0/30000
nll_loss: 314.322, prior_loss: 2506.492, total: 2820.813
noise: 0.123
epoch:   100/30000
nll_loss: 267.270, prior_loss: 2723.424, total: 2990.694
noise: 0.123
epoch:   200/30000
nll_loss: 262.959, prior_loss: 2835.986, total: 3098.945
noise: 0.123
epoch:   300/30000
nll_loss: 259.089, prior_loss: 2932.834, total: 3191.923
noise: 0.123
epoch:   400/30000
nll_loss: 254.036, prior_loss: 3023.945, total: 3277.981
noise: 0.123
epoch:   500/30000
nll_loss: 249.374, prior_loss: 3094.060, total: 3343.433
noise: 0.123
epoch:   600/30000
nll_loss: 234.570, prior_loss: 3174.145, total: 3408.715
noise: 0.123
epoch:   700/30000
nll_loss: 219.074, prior_loss: 3259.439, total: 3478.513
noise: 0.123
epoch:   800/30000
nll_loss: 216.232, prior_loss: 3323.198, total: 3539.431
noise: 0.123
epoch:   900/30000
nll_loss: 205.812, prior_loss: 3382.825, total: 3588.638
noise: 0.123
epoch:  1000/30000
nll_loss: 197.376, prior_loss: 3451.892, total: 3649.268
noise: 0.123
epoch:  1100/30000
nl